In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from peakutils import indexes
from  scipy.signal import find_peaks as fp

from bokeh.plotting import figure , show

from bokeh.io import output_notebook
from bokeh.models import LabelSet

output_notebook()


Loading BokehJS ...

# Importing the original Spectra Data which we have

In [3]:
spectra_data_og = pd.read_csv(r'Spectra Data.csv', skiprows=0 ,usecols = [1,2,3,4,5,6,7,8] )
spectra_data_og

,CR210LA,CR240LA,CR300LA,CR570CP,CR440Y,CR700Y,CR1000Y,HR660Y
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
23426,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23427,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23428,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23429,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Stripping the first columns , what we have now is all the intensities for all the sample.

In [4]:
spectra_data = pd.read_csv(r'Spectra Data.csv', skiprows=0 , usecols=[1,2,3,4,5,6,7,8])
spectra_data_trans = spectra_data.transpose()
spectra_data_trans

,0,1,2,3,4,5,6,7,8,9,...,23421,23422,23423,23424,23425,23426,23427,23428,23429,23430
CR210LA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CR240LA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CR300LA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CR570CP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CR440Y,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CR700Y,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CR1000Y,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HR660Y,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Importing the ORIGINAL concentration file which we have

In [5]:
concentration_data_og = pd.read_csv(r'Concentration.csv', skiprows=0)
concentration_data_og

,wavelength,CR210LA,CR240LA,CR300LA,CR570CP,CR440Y,CR700Y,CR1000Y,HR660Y
0,Fe,99.00,99.00,99.00,97.0,97.0,96.0,98.0,97.0
1,Cu,0.10,0.10,0.10,0.1,0.1,0.1,0.1,0.1
2,Mo,0.00,0.05,0.05,0.5,0.6,0.5,0.3,0.5
3,Ni,0.05,0.05,0.10,0.7,0.1,0.0,0.0,0.1
4,Al,0.00,0.10,0.05,0.5,0.8,0.5,0.2,0.5
5,Si,0.20,0.20,0.30,0.5,0.4,0.5,0.2,0.5
6,Cr,0.00,0.05,0.05,0.5,0.7,0.5,0.4,0.5
7,Mn,0.30,0.40,0.60,1.2,1.2,1.2,1.2,1.0


In [6]:
concentration_data = pd.read_csv(r'Concentration.csv', skiprows=0 , usecols= [1,2,3,4,5,6,7,8])
concentration_data_trans = concentration_data.transpose()
concentration_data_trans

,0,1,2,3,4,5,6,7
CR210LA,99.0,0.1,0.00,0.05,0.00,0.2,0.00,0.3
CR240LA,99.0,0.1,0.05,0.05,0.10,0.2,0.05,0.4
CR300LA,99.0,0.1,0.05,0.10,0.05,0.3,0.05,0.6
CR570CP,97.0,0.1,0.50,0.70,0.50,0.5,0.50,1.2
CR440Y,97.0,0.1,0.60,0.10,0.80,0.4,0.70,1.2
CR700Y,96.0,0.1,0.50,0.00,0.50,0.5,0.50,1.2
CR1000Y,98.0,0.1,0.30,0.00,0.20,0.2,0.40,1.2
HR660Y,97.0,0.1,0.50,0.10,0.50,0.5,0.50,1.0


# Importing Regression Models

In [7]:
from sklearn import linear_model
from sklearn.metrics import r2_score
reg = linear_model.LinearRegression()

# For each values of rows of spectradata from 0 -- 23430 , I will build a regression fit for each of those 23431 values.      The Regession fit will take Intensities at X and Actual Concentration of data at Y

In [8]:
# 0 = Fe , 1 = Cu , 2 = Mo , 3 = Ni , 4 = Al , 5 = Si , 6 = Cr , 7 = Mn

reference_actual_concentration = concentration_data_trans[2].values
reference_actual_concentration = np.array(reference_actual_concentration).reshape(-1,1)

#print(reference_actual_concentration)

######################################################################################################
######################################################################################################

score = [[],[]]

for iteam in spectra_data_trans.columns:
    intensities = spectra_data_trans[iteam].values
    intensities = np.array(intensities).reshape(-1,1)
    reg.fit(intensities , reference_actual_concentration)
    r2 = reg.score(intensities , reference_actual_concentration)
    if r2 > 0.7:
        print(r2 , iteam )

0.7045154903104738 414
0.7067434076010618 415
0.7016975579636366 416
0.7065993001882167 417
0.7067367994575278 566
0.7026613313155252 1212
0.7246941957969412 1213
0.7212154794409159 1214
0.7325755318796201 6427
0.7358310215745302 6436
0.8052692238334281 6437
0.831003088231361 6438
0.7531001797329961 6439
0.7587564634568363 6477
0.7410746176919041 6478
0.7114355337572265 6479
0.7553304326221104 6490
0.7424719468391134 6491
